# Biology - Digital World 2D

Cohort: 08

Group Number: 08

Members
* Ang Wei Jie             1002815
* Nashita Abd Guntaguli   1003045
* Emil Goh Pei Peng       1002762
* Keerthana Janmugam      1003079
* Jayvan Leong Ghit       1002780
    

In [1]:
import random
import math
from fractions import Fraction
import numpy as np


## Introduction

**Aim: To generate suitable primers with a score of objective function close to 0 to optimize PCR and make it an efficient process.**

Running all cells initially will display results for each of the functions present in the class.

### Initializing Parameters
Efficient primer design is essential for successful reactions. In the process of PCR, design considerations and parameters are important for amplifications with high yield. The parameters specified below correspond to optimum values that enhance the process of annealing by producing accurate choice of primers.

### Specifying Penalty Parameters
Penalty parameters have been specified in accordance with the importance and magnitude of cost of respective criterions.



In [2]:
class PrimerDesign(object):
    
    def __init__ (self, name):
        
        '''parameters for the length criterion'''
        self.max_length = 22
        self.min_length = 18
        self.penalty_length = 1
        
        '''parameters for the temperature difference criterion'''
        self.max_tdiff = 4
        self.min_tdiff = 0
        self.penalty_tdiff = 1
        
        '''parameters for the cg content criterion'''
        self.max_cg = 0.6
        self.min_cg = 0.4
        self.penalty_cg = 2
        
        '''parameters for the annealing temperature criterion'''
        self.max_temp = 65
        self.min_temp = 45
        self.penalty_temp = 1
        
        '''parameters for the run criterion'''
        self.run_threshold = 4
        self.penalty_runs = 1
        
        '''parameters for the repeat criterion'''
        self.repeat_threshold = 0
        self.penalty_repeats = 1
        
        '''parameters for the specificity criterion'''
        self.penalty_specificity = 1
        
        '''locations where the forward primer should be chosen from'''
        self.fp_start = 1000
        self.fp_end = 2000
        
        '''locations where the reverse primer should be chosen from'''
        self.rp_start = 3000
        self.rp_end = 4000
        
        ''' parameters for the simulated annealing portion'''
        self.initial_temperature = 200
        self.stopping_temperature = 0.1
        self.drop_fraction = 0.95
        

In [3]:
allele1 = '''1 gtccgggcag cccccggcgc agcgcggccg cagcagcctc cgccccccgc acggtgtgag
61 cgcccgacgc ggccgaggcg gccggagtcc cgagctagcc ccggcggccg ccgccgccca
121 gaccggacga caggccacct cgtcggcgtc cgcccgagtc cccgcctcgc cgccaacgcc
181 acaaccaccg cgcacggccc cctgactccg tccagtattg atcgggagag ccggagcgag
241 ctcttcgggg agcagcgatg cgaccctccg ggacggccgg ggcagcgctc ctggcgctgc
301 tggctgcgct ctgcccggcg agtcgggctc tggaggaaaa gaaagtttgc caaggcacga
361 gtaacaagct cacgcagttg ggcacttttg aagatcattt tctcagcctc cagaggatgt
421 tcaataactg tgaggtggtc cttgggaatt tggaaattac ctatgtgcag aggaattatg
481 atctttcctt cttaaagacc atccaggagg tggctggtta tgtcctcatt gccctcaaca
541 cagtggagcg aattcctttg gaaaacctgc agatcatcag aggaaatatg tactacgaaa
601 attcctatgc cttagcagtc ttatctaact atgatgcaaa taaaaccgga ctgaaggagc
661 tgcccatgag aaatttacag gaaatcctgc atggcgccgt gcggttcagc aacaaccctg
721 ccctgtgcaa cgtggagagc atccagtggc gggacatagt cagcagtgac tttctcagca
781 acatgtcgat ggacttccag aaccacctgg gcagctgcca aaagtgtgat ccaagctgtc
841 ccaatgggag ctgctggggt gcaggagagg agaactgcca gaaactgacc aaaatcatct
901 gtgcccagca gtgctccggg cgctgccgtg gcaagtcccc cagtgactgc tgccacaacc
961 agtgtgctgc aggctgcaca ggccccatgg agagcgactg cctggtctgc cgcaaattcc
1021 gagacgaagc cacgtgcaag gacacctgcc ccccactcat gctctacaac cccaccacgt
1081 accagatgga tgtgaacccc gagggcaaat acagctttgg tgccacctgc gtgaagaagt
1141 gtccccgtaa ttatgtggtg acagatcacg gctcgtgcgt ccgagcctgt ggggccgaca
1201 gctatgagat ggaggaagac ggcgtccgca agtgtaagaa gtgcgaaggg ccttgccgca
1261 aagtgtgtaa cggaataggt attggtgaat ttaaagactc actctccata aatgctacga
1321 atattaaaca cttcaaaaac tgcacctcca tcagtggcga tctccacatc ctgccggtgg
1381 catttagggg tgactccttc acacatactc ctcctctgga tccacaggaa ctggatattc
1441 tgaaaaccgt aaaggaaatc acagggtttt tgctgattca ggcttggcct gaaaacagga
1501 cggacctcca tgcctttgag aacctagaaa tcatacgcgg caggaccaag caacatggtc
1561 agttttctct tgcagtcgtc agcctgaaca taacatcctt gggattacgc tccctcaagg
1621 agataagtga tggagatgtg ataatttcag gaaacaaaaa tttgtgctat gcaaatacaa
1681 taaactggaa aaaactgttt gggacctccg gtcagaaaac caaaattata agcaacagag
1741 gtgaaaacag ctgcaaggcc acaggccagg tctgccatgc cttgtgctcc cccgagggct
1801 gctggggccc ggagcccagg gactgcgtct cttgccggaa tgtcagccga ggcagggaat
1861 gcgtggacaa gtgcaacctt ctggagggtg agccaaggga gtttgtggag aactctgagt
1921 gcatacagtg ccacccagag tgcctgcctc aggccatgaa catcacctgc acaggacggg
1981 gaccagacaa ctgtatccag tgtgcccact acattgacgg cccccactgc gtcaagacct
2041 gcccggcagg agtcatggga gaaaacaaca ccctggtctg gaagtacgca gacgcttgcc
2101 atgtgtgcca cctgtgccat ccaaactgca cctacggatg cactgggcca ggtcttgaag
2161 gctgtccaac gaatgggcct aagatcccgt ccatcgccac tgggatggtg ggggccctcc
2221 tcttgctgct ggtggtggcc ctggggatcg gcctcttcat gcgaaggcgc cacatcgttc
2281 ggaagcgcac gctgcggagg ctgctgcagg agagggagct tgtggagcct cttacaccca
2341 gtggagaagc tcccaaccaa gctctcttga ggatcttgaa ggaaactgaa ttcaaaaaga
2401 tcaaagtgct gggctccggt gcgttcggca cggtgtataa gggactctgg atcccagaag
2461 gtgagaaagt taaaattccc gtcgctatca aggaattaag agaagcaaca tctccgaaag
2521 ccaacaagga aatcctcgat gaagcctacg tgatggccag cgtggacaac ccccacgtgt
2581 gccgcctgct gggcatctgc ctcacctcca ccgtgcagct catcacgcag ctcatgccct
2641 tcggctgcct cctggactat gtccgggaac acaaagacaa tattggctcc cagtacctgc
2701 tcaactggtg tgtgcagatc gcaaagggca tgaactactt ggaggaccgt cgcttggtgc
2761 accgcgacct ggcagccagg aacgtactgg tgaaaacacc gcagcatgtc aagatcacag
2821 attttgggct ggccaaactg ctgggtgcgg aagagaaaga ataccatgca gaaggaggca
2881 aagtgcctat caagtggatg gcattggaat caattttaca cagaatctat acccaccaga
2941 gtgatgtctg gagctacggg gtgactgttt gggagttgat gacctttgga tccaagccat
3001 atgacggaat ccctgccagc gagatctcct ccatcctgga gaaaggagaa cgcctccctc
3061 agccacccat atgtaccatc gatgtctaca tgatcatggt caagtgctgg atgatagacg
3121 cagatagtcg cccaaagttc cgtgagttga tcatcgaatt ctccaaaatg gcccgagacc
3181 cccagcgcta ccttgtcatt cagggggatg aaagaatgca tttgccaagt cctacagact
3241 ccaacttcta ccgtgccctg atggatgaag aagacatgga cgacgtggtg gatgccgacg
3301 agtacctcat cccacagcag ggcttcttca gcagcccctc cacgtcacgg actcccctcc
3361 tgagctctct gagtgcaacc agcaacaatt ccaccgtggc ttgcattgat agaaatgggc
3421 tgcaaagctg tcccatcaag gaagacagct tcttgcagcg atacagctca gaccccacag
3481 gcgccttgac tgaggacagc atagacgaca ccttcctccc agtgcctgaa tacataaacc
3541 agtccgttcc caaaaggccc gctggctctg tgcagaatcc tgtctatcac aatcagcctc
3601 tgaaccccgc gcccagcaga gacccacact accaggaccc ccacagcact gcagtgggca
3661 accccgagta tctcaacact gtccagccca cctgtgtcaa cagcacattc gacagccctg
3721 cccactgggc ccagaaaggc agccaccaaa ttagcctgga caaccctgac taccagcagg
3781 acttctttcc caaggaagcc aagccaaatg gcatctttaa gggctccaca gctgaaaatg
3841 cagaatacct aagggtcgcg ccacaaagca gtgaatttat tggagcatga ccacggagga
3901 tagtatgagc cctaaaaatc cagactcttt cgatacccag gaccaagcca cagcaggtcc
3961 tccatcccaa cagccatgcc cgcattagct cttagaccca cagactggtt ttgcaacgtt
4021 tacaccgact agccaggaag tacttccacc tcgggcacat tttgggaagt tgcattcctt
4081 tgtcttcaaa ctgtgaagca tttacagaaa cgcatccagc aagaatattg tccctttgag
4141 cagaaattta tctttcaaag aggtatattt gaaaaaaaaa aaaagtatat gtgaggattt
4201 ttattgattg gggatcttgg agtttttcat tgtcgctatt gatttttact tcaatgggct
4261 cttccaacaa ggaagaagct tgctggtagc acttgctacc ctgagttcat ccaggcccaa
4321 ctgtgagcaa ggagcacaag ccacaagtct tccagaggat gcttgattcc agtggttctg
4381 cttcaaggct tccactgcaa aacactaaag atccaagaag gccttcatgg ccccagcagg
4441 ccggatcggt actgtatcaa gtcatggcag gtacagtagg ataagccact ctgtcccttc
4501 ctgggcaaag aagaaacgga ggggatggaa ttcttcctta gacttacttt tgtaaaaatg
4561 tccccacggt acttactccc cactgatgga ccagtggttt ccagtcatga gcgttagact
4621 gacttgtttg tcttccattc cattgttttg aaactcagta tgctgcccct gtcttgctgt
4681 catgaaatca gcaagagagg atgacacatc aaataataac tcggattcca gcccacattg
4741 gattcatcag catttggacc aatagcccac agctgagaat gtggaatacc taaggatagc
4801 accgcttttg ttctcgcaaa aacgtatctc ctaatttgag gctcagatga aatgcatcag
4861 gtcctttggg gcatagatca gaagactaca aaaatgaagc tgctctgaaa tctcctttag
4921 ccatcacccc aaccccccaa aattagtttg tgttacttat ggaagatagt tttctccttt
4981 tacttcactt caaaagcttt ttactcaaag agtatatgtt ccctccaggt cagctgcccc
5041 caaaccccct ccttacgctt tgtcacacaa aaagtgtctc tgccttgagt catctattca
5101 agcacttaca gctctggcca caacagggca ttttacaggt gcgaatgaca gtagcattat
5161 gagtagtgtg gaattcaggt agtaaatatg aaactagggt ttgaaattga taatgctttc
5221 acaacatttg cagatgtttt agaaggaaaa aagttccttc ctaaaataat ttctctacaa
5281 ttggaagatt ggaagattca gctagttagg agcccacctt ttttcctaat ctgtgtgtgc
5341 cctgtaacct gactggttaa cagcagtcct ttgtaaacag tgttttaaac tctcctagtc
5401 aatatccacc ccatccaatt tatcaaggaa gaaatggttc agaaaatatt ttcagcctac
5461 agttatgttc agtcacacac acatacaaaa tgttcctttt gcttttaaag taatttttga
5521 ctcccagatc agtcagagcc cctacagcat tgttaagaaa gtatttgatt tttgtctcaa
5581 tgaaaataaa actatattca tttccactct attatgctct caaatacccc taagcatcta
5641 tactagcctg gtatgggtat gaaagataca aagataaata aaacatagtc cctgattcta
5701 agaaattcac aatttagcaa aggaaatgga ctcatagatg ctaaccttaa aacaacgtga
5761 caaatgccag acaggaccca tcagccaggc actgtgagag cacagagcag ggaggttggg
5821 tcctgcctga ggagacctgg aagggaggcc tcacaggagg atgaccaggt ctcagtcagc
5881 ggggaggtgg aaagtgcagg tgcatcaggg gcaccctgac cgaggaaaca gctgccagag
5941 gcctccactg ctaaagtcca cataaggctg aggtcagtca ccctaaacaa cctgctccct
6001 ctaagccagg ggatgagctt ggagcatccc acaagttccc taaaagttgc agcccccagg
6061 gggattttga gctatcatct ctgcacatgc ttagtgagaa gactacacaa catttctaag
6121 aatctgagat tttatattgt cagttaacca ctttcattat tcattcacct caggacatgc
6181 agaaatattt cagtcagaac tgggaaacag aaggacctac attctgctgt cacttatgtg
6241 tcaagaagca gatgatcgat gaggcaggtc agttgtaagt gagtcacatt gtagcattaa
6301 attctagtat ttttgtagtt tgaaacagta acttaataaa agagcaaaag ctaaaaaaaa
6361 aaaaaaaaa
'''
dna_sequence = allele1

#### Gene Sequences Available:
dna_sequence,
allele1,
allele2

Currently, allele1 = dna_sequence


### Producing DNA sequence
The function below cleans the raw gene sequence to leave behind only the bases a, t, c, g.
Numbers and spaces present in the raw gene sequence have been eliminated.


In [4]:
class PrimerDesign(PrimerDesign): 
    
    def set_dna_sequence(self, dna_sequence):
        self.dna_sequence = str(dna_sequence)
        ls = list(dna_sequence)
        lsnew = []
        for i in ls:
            if str.isalpha(i) == True:
                lsnew.append(i)
    
            else:
                pass
        self.dna_sequence = "".join(lsnew)
        #dna_sequence = dna_sequence
        return self.dna_sequence
        
print(PrimerDesign(PrimerDesign).set_dna_sequence(dna_sequence))
#print(c.set_dna_sequence(allele2))

gtccgggcagcccccggcgcagcgcggccgcagcagcctccgccccccgcacggtgtgagcgcccgacgcggccgaggcggccggagtcccgagctagccccggcggccgccgccgcccagaccggacgacaggccacctcgtcggcgtccgcccgagtccccgcctcgccgccaacgccacaaccaccgcgcacggccccctgactccgtccagtattgatcgggagagccggagcgagctcttcggggagcagcgatgcgaccctccgggacggccggggcagcgctcctggcgctgctggctgcgctctgcccggcgagtcgggctctggaggaaaagaaagtttgccaaggcacgagtaacaagctcacgcagttgggcacttttgaagatcattttctcagcctccagaggatgttcaataactgtgaggtggtccttgggaatttggaaattacctatgtgcagaggaattatgatctttccttcttaaagaccatccaggaggtggctggttatgtcctcattgccctcaacacagtggagcgaattcctttggaaaacctgcagatcatcagaggaaatatgtactacgaaaattcctatgccttagcagtcttatctaactatgatgcaaataaaaccggactgaaggagctgcccatgagaaatttacaggaaatcctgcatggcgccgtgcggttcagcaacaaccctgccctgtgcaacgtggagagcatccagtggcgggacatagtcagcagtgactttctcagcaacatgtcgatggacttccagaaccacctgggcagctgccaaaagtgtgatccaagctgtcccaatgggagctgctggggtgcaggagaggagaactgccagaaactgaccaaaatcatctgtgcccagcagtgctccgggcgctgccgtggcaagtcccccagtgactgctgccacaaccagtgtgctgcaggctgcacaggccccatggagagcgactg

### Generating Primers
The function below, func_select_random selects primers of specified length from the dna sequence within the instantiated range self.fp/rp_start and self.fp/rp_end. A random position is selected in the given range and DNA bases are added to the random location for specified length. It produces both, the gene sequence for forward and reverse primers based on specifications made in function parameters. 

The 'complement' function produces the complementary strand to the specified gene sequence.

In [5]:
class PrimerDesign(PrimerDesign):
    
    def func_select_random(self, sqtype, length = 20):
        
        '''the length has to be a positive number'''
        ls_fp = []
        ls_rp = []
        dna = PrimerDesign(PrimerDesign).set_dna_sequence(dna_sequence)
        if(sqtype == 'forward'):
            #print(self.fp_start)
            start_limit = self.fp_start 
            end_limit = self.fp_end 
            x = random.randint(start_limit, end_limit)
            
            for i in range(x, length+x):
                z = dna[i]
                ls_fp.append(z)
                fp = "".join(ls_fp)
            return fp
        
        elif(sqtype == 'reverse'):
        
            start_limit = self.rp_start 
            end_limit = self.rp_end
            x = random.randint(start_limit, end_limit)
            
            for i in range(x, length+x):
                m = dna[i]
                ls_rp.append(m)
                rp = "".join(ls_rp)
            return rp
        
        else: 
            return None
        
    def complement(self, sq):
        ls = []
        for i in sq:
            if i == 'c':
                ls.append('g')
            elif i == 'g':
                ls.append('c')
            elif i == 'a':
                ls.append('t')
            elif i == 't':
                ls.append('a')
        compl = "".join(ls)
        return compl

fp = PrimerDesign(PrimerDesign).func_select_random(sqtype='forward', length = 20)
rp = PrimerDesign(PrimerDesign).func_select_random(sqtype='reverse', length = 20 )
print(fp, rp)

cof = PrimerDesign(PrimerDesign).complement(fp)
cor = PrimerDesign(PrimerDesign).complement(rp)
print(cof, cor)

tccggtcagaaaaccaaaat aaaggagaacgcctccctca
aggccagtcttttggtttta tttcctcttgcggagggagt


### Calculating Primer Properties
Properties of the selected PCR primers or gene sequences are calculated using the following functions.
* func_length : Length of the primer = Number of bases in the primer


* func_cg_fraction : Content of bases 'c' and 'g' in the primer
* func_cg_fraction_toprint : Percentage Content of bases 'c' and 'g' in the primer


* func_at_fraction : Content of bases 'a' and 't' in the primer
* func_at_fraction_toprint : Percentage Content of bases 'a' and 't' in the primer


* func_temperature : The annealing temperature required for the primer, calculated using the formula, T = 4(Cs+Gs)+2(As+Ts)

In [6]:
class PrimerDesign(PrimerDesign): 
    
    def func_length(self, sq):
        length = len(sq) 
        return length
    
    def func_cg_fraction(self, sq):
        sq = str(sq)
        length = len(sq)
        c = sq.count("c")
        g = sq.count("g")
        #fraction = "{}/{}".format((c+g),length)
        if length == 0:
            fraction = 0
        else:
            frac = (c+g)/length
            fraction = Fraction(frac).limit_denominator(length)
        return fraction
    
    def func_cg_fraction_toprint(self, sq):
        sq = str(sq)
        length = len(sq)
        c = sq.count("c")
        g = sq.count("g")
        #fraction = "{}/{}".format((c+g),length)
        frac = float((c+g)/length*100)
        return round(frac,1)
    
    def func_at_fraction(self, sq):
        sq = str(sq)
        length = len(sq)
        a = sq.count("a")
        t = sq.count("t")
        #fraction = "{}/{}".format((a+t),length)
        frac = (a+t)/length
        fraction = Fraction(frac).limit_denominator(length)
        return fraction
    
    def func_at_fraction_toprint(self, sq):
        sq = str(sq)
        length = len(sq)
        a = sq.count("a")
        t = sq.count("t")
        #fraction = "{}/{}".format((c+g),length)
        frac = (a+t)/length*100
        return round(frac,1)
    
    
    def func_temperature(self,sq):
        sq = str(sq)
        g = sq.count("g")
        c = sq.count("c")
        t = sq.count("t")
        a = sq.count("a")
        temp = (4*(c+g))+(2*(a+t))
        return temp

print(PrimerDesign(PrimerDesign).func_length(fp),PrimerDesign(PrimerDesign).func_length(rp))
print(PrimerDesign(PrimerDesign).func_cg_fraction(fp),PrimerDesign(PrimerDesign).func_cg_fraction(rp))
print(PrimerDesign(PrimerDesign).func_cg_fraction_toprint(fp),PrimerDesign(PrimerDesign).func_cg_fraction_toprint(rp))
print(PrimerDesign(PrimerDesign).func_temperature(fp),PrimerDesign(PrimerDesign).func_temperature(rp))

20 20
2/5 11/20
40.0 55.0
56 62


### Calculating Number of Runs and Repeats
func_count_runs returns the number of consecutive runs greater than Rmax = 4 for a given gene sequence or primer.
func_count_repeats returns the number of times a sequence of 2 bases has been repeated consecutively.

In [7]:
class PrimerDesign(PrimerDesign):

    def func_count_runs(self, sq):
        sq = str(sq)
        count=1
        ls = []
        for i in range(1,len(sq)):
            
            if sq[i-1]==sq[i]:
               count = count + 1
            
            else :
                
                ls.append(count)
                count=1
        x=0
        for m in ls:
            if m>4:
                x+=1
        return x
    
print(PrimerDesign(PrimerDesign).func_count_runs(fp),PrimerDesign(PrimerDesign).func_count_runs(rp))

0 0


In [8]:
class PrimerDesign(PrimerDesign):
    def func_count_repeats(self, sq):
        di_repeats = ['at','ac','ag','ca','ct','cg','ga','gt','gc','ta','tc','tg']
        l = len(sq)
        repeats = 0
        i = 0
        while i+3 < l:
            if sq[i]+sq[i+1] in di_repeats and sq[i]+sq[i+1]==sq[i+2]+sq[i+3]:
                x = 1
                while sq[i]+sq[i+1] in di_repeats and sq[i]+sq[i+1]==sq[i+2]+sq[i+3]:
                    x+=1
                    i+=1
                    if i+3>=l-1:
                        break
                repeats += x - 1
            else:
                i+=1
        return repeats
    
print(PrimerDesign(PrimerDesign).func_count_repeats(fp),PrimerDesign(PrimerDesign).func_count_repeats(rp))

0 1


### Cost Functions
Properties of primers for PCR can be implemented as cost functions to determine the suitability of the primers for optimizing PCR. Cost Functions return 0 if the criterion is met.

* cost_length: depends on length of primer between the optimum maximum and optimum minimum length
* cost_temperature: depends on annealing temperature of primer between the optimum maximum and optimum minimum temperature
* cost_cgcontent: GC content must be between 40-60%
* cost_temperature_difference: difference in primer annealing temperature of both primers ∆𝑇 must be below maximum Temperature difference
* cost_specificity: primers ought to be specific i.e. primers can only bond to one position on the dna sequence.
* cost_runs: runs of the same nucleotide are avoided. A maximum of 4 base runs is acceptable.
* cost_repeats: repeating sequences of two bases in the primer must be avoided

In [9]:
class PrimerDesign(PrimerDesign):
    
    def cost_length(self, sq):
        '''This is given to you as an example '''
        sq_len = len(sq)
        if(sq_len > self.max_length):
            return (sq_len - self.max_length)*self.penalty_length
        elif(sq_len > self.min_length):
            return 0
        else:
            return (self.min_length - sq_len)*self.penalty_length 
    
    def cost_temperature(self, sq):
        sq_temp = PrimerDesign(PrimerDesign).func_temperature(sq)
        
        if(sq_temp > self.max_temp):
            return (sq_temp - self.max_temp)*self.penalty_temp
        elif(sq_temp > self.min_temp):
            return 0
        else:
            return (self.min_temp - sq_temp)*self.penalty_temp
        
    def cost_cgcontent(self,sq):
        sq_cg = PrimerDesign(PrimerDesign).func_cg_fraction(sq)
        
        if(sq_cg > self.max_cg):
            return round((sq_cg - self.max_cg)*self.penalty_cg, 2)
        elif(sq_cg > self.min_cg):
            return 0
        else:
            return round((self.min_cg - sq_cg)*self.penalty_cg, 2)
        
    def cost_cgcontent_toprint(self,sq):
        sq_cg = PrimerDesign(PrimerDesign).cost_cgcontent(sq)
        sq_cg = float(sq_cg)
        return sq_cg
        
    def cost_temperature_difference(self, fp, rp):
        t_fp = PrimerDesign(PrimerDesign).func_temperature(fp)
        t_rp = PrimerDesign(PrimerDesign).func_temperature(rp)
        _t = abs(t_fp - t_rp)
        if _t > self.max_tdiff:
            cost = (_t - self.max_tdiff)*self.penalty_tdiff
        else:
            cost = 0
        return cost
    
    def cost_specificity(self, sq):
        #comp = PrimerDesign(PrimerDesign).complement(sq)
        dna = PrimerDesign(PrimerDesign).set_dna_sequence(dna_sequence)
        n = dna.count(sq)
        cost =(n-1)*self.penalty_specificity
        return cost
        
    def cost_runs(self, sq):
        x = PrimerDesign(PrimerDesign).func_count_runs(sq)
        #print(x)
        return self.penalty_runs*x

    def cost_repeats(self,sq):
        x = PrimerDesign(PrimerDesign).func_count_repeats(sq)
        return self.penalty_repeats*x

print(PrimerDesign(PrimerDesign).cost_length(fp))
print(PrimerDesign(PrimerDesign).cost_temperature(fp))
print(PrimerDesign(PrimerDesign).cost_cgcontent(fp))
print(PrimerDesign(PrimerDesign).cost_temperature_difference(fp, rp))
print(PrimerDesign(PrimerDesign).cost_specificity(fp))
print(PrimerDesign(PrimerDesign).cost_runs(fp))
print(PrimerDesign(PrimerDesign).cost_repeats(fp))

0
0
0.0
2
0
0
0


### Objective Function
cost_objective_function calculates the cost of the all the properties together. It adds the cost functions mentioned above for both, the forward and reverse primer. A score close to zero indicates that the primers selected are close to the optimal choice.

In [10]:
class PrimerDesign(PrimerDesign):
    
    def cost_objective_function(self, fp, rp):
        '''complete the calculation of the cost'''
        cost = 0      
        cost = PrimerDesign(PrimerDesign).cost_temperature(fp)+PrimerDesign(PrimerDesign).cost_temperature(rp)+PrimerDesign(PrimerDesign).cost_length(fp)+PrimerDesign(PrimerDesign).cost_length(rp)+PrimerDesign(PrimerDesign).cost_cgcontent_toprint(fp)+PrimerDesign(PrimerDesign).cost_cgcontent_toprint(rp)+PrimerDesign(PrimerDesign).cost_specificity(fp)+PrimerDesign(PrimerDesign).cost_specificity(rp)+PrimerDesign(PrimerDesign).cost_runs(fp)+PrimerDesign(PrimerDesign).cost_runs(rp)+PrimerDesign(PrimerDesign).cost_repeats(fp)+PrimerDesign(PrimerDesign).cost_repeats(rp)+PrimerDesign(PrimerDesign).cost_temperature_difference(fp, rp)
        return cost 
print(PrimerDesign(PrimerDesign).cost_objective_function(fp, rp))

3.0


### Primers and their Properties
The following function displays a report containing all required information of selected primers.

In [11]:
class PrimerDesign(PrimerDesign):
    
    def cost_objective_function_info(self, fp, rp):
        cf = PrimerDesign(PrimerDesign).complement(fp)
        cr = PrimerDesign(PrimerDesign).complement(rp)
        l_fp = PrimerDesign(PrimerDesign).func_length(fp)
        l_rp = PrimerDesign(PrimerDesign).func_length(rp)
        c_l_fp = PrimerDesign(PrimerDesign).cost_length(fp)
        c_l_rp = PrimerDesign(PrimerDesign).cost_length(rp)
        
        t_fp = PrimerDesign(PrimerDesign).func_temperature(fp)
        t_rp = PrimerDesign(PrimerDesign).func_temperature(rp) 
        c_t_fp = PrimerDesign(PrimerDesign).cost_temperature(fp)
        c_t_rp = PrimerDesign(PrimerDesign).cost_temperature(rp)
        
        cg_fp = PrimerDesign(PrimerDesign).func_cg_fraction_toprint(fp)
        cg_rp = PrimerDesign(PrimerDesign).func_cg_fraction_toprint(rp)
        c_cg_fp = PrimerDesign(PrimerDesign).cost_cgcontent_toprint(fp)
        c_cg_rp = PrimerDesign(PrimerDesign).cost_cgcontent_toprint(rp)
        at_fp = PrimerDesign(PrimerDesign).func_at_fraction_toprint(fp)
        at_rp = PrimerDesign(PrimerDesign).func_at_fraction_toprint(rp)

        print("{0:^60}".format("Selected Primers"))
        print("{0:^60}".format("===Forward Primer=== {}".format(cf)))
        print("\n")
        print("{0:<20}".format("Criterion")+"{0:^20}".format("Value")+"{0:>20}".format("Cost Function Score"))
        print("{0:-^60}".format("-"))
        print("{0:<20}".format("Length")+"{0:^20}".format(l_fp)+"{0:>20}".format(c_l_fp))
        print("{0:<21}".format("Annealing Temperature")+"{0:^19}".format(t_fp)+"{0:>20}".format(c_t_fp))
        print("{0:<20}".format("% cg Content")+"{0:^20}".format(cg_fp)+"{0:>20}".format(c_cg_fp))
        print("{0:<20}".format("% at Content")+"{0:^20}".format(at_fp))
        print("{0:<20}".format("Runs")+"{0:^20}".format(PrimerDesign(PrimerDesign).func_count_runs(fp))+"{0:>20}".format(PrimerDesign(PrimerDesign).cost_runs(fp)))
        print("{0:<20}".format("Repeats")+"{0:^20}".format(PrimerDesign(PrimerDesign).func_count_repeats(fp))+"{0:>20}".format(PrimerDesign(PrimerDesign).cost_repeats(fp)))
        print("{0:<30}".format("Specificity")+"{0:>30}".format(PrimerDesign(PrimerDesign).cost_specificity(fp)))
        print("\n")
        print("{0:^60}".format("===Reverse Primer=== {}".format(cr)))
        print("\n")
        print("{0:<20}".format("Criterion")+"{0:^20}".format("Value")+"{0:>20}".format("Cost Function Score"))
        print("{0:-^60}".format("-"))
        print("{0:<20}".format("Length")+"{0:^20}".format(l_rp)+"{0:>20}".format(c_l_rp))
        print("{0:<21}".format("Annealing Temperature")+"{0:^19}".format(t_rp)+"{0:>20}".format(c_t_rp))
        print("{0:<20}".format("% cg Content")+"{0:^20}".format(cg_rp)+"{0:>20}".format(c_cg_rp))
        print("{0:<20}".format("% at Content")+"{0:^20}".format(at_rp))
        print("{0:<20}".format("Runs")+"{0:^20}".format(PrimerDesign(PrimerDesign).func_count_runs(rp))+"{0:>20}".format(PrimerDesign(PrimerDesign).cost_runs(rp)))
        print("{0:<20}".format("Repeats")+"{0:^20}".format(PrimerDesign(PrimerDesign).func_count_repeats(rp))+"{0:>20}".format(PrimerDesign(PrimerDesign).cost_repeats(rp)))
        print("{0:<30}".format("Specificity")+"{0:>30}".format(PrimerDesign(PrimerDesign).cost_specificity(rp)))
        print("\n")
        #print("{0:-^60}".format("-"))
        print("{0:<30}".format("Temperature Difference")+"{0:>30}".format(PrimerDesign(PrimerDesign).cost_temperature_difference(fp,rp)))
        print("{0:-^60}".format("-"))
        print("{0:<30}".format("Objective Function")+"{0:>30}".format(PrimerDesign(PrimerDesign).cost_objective_function(fp,rp)))
        print("{0:-^60}".format("-"))
        print("\n")

PrimerDesign(PrimerDesign).cost_objective_function_info(fp, rp)       


                      Selected Primers                      
         ===Forward Primer=== aggccagtcttttggtttta          


Criterion                  Value         Cost Function Score
------------------------------------------------------------
Length                       20                            0
Annealing Temperature        56                            0
% cg Content                40.0                         0.0
% at Content                60.0        
Runs                         0                             0
Repeats                      0                             0
Specificity                                                0


         ===Reverse Primer=== tttcctcttgcggagggagt          


Criterion                  Value         Cost Function Score
------------------------------------------------------------
Length                       20                            0
Annealing Temperature        62                            0
% cg Content                55.0      

### Simulated Annealing Algorithm

In [12]:
class PrimerDesign(PrimerDesign): 
    
    def func_simulated_annealing(self):
        
        temperature = self.initial_temperature
        stopping_temperature = self.stopping_temperature
        drop = self.drop_fraction
        
        fp_current = PrimerDesign(PrimerDesign).func_select_random(sqtype='forward', length = 20) #generates random sequence of forward primer of length 20 bases
        rp_current = PrimerDesign(PrimerDesign).func_select_random(sqtype='reverse', length = 20) #generates random sequence of reverse primer of length 20 bases
        #print(rp_current)
        cost = PrimerDesign(PrimerDesign).cost_objective_function(fp_current, rp_current) #calculates cost of generated primers
        current_cost = cost
        i = 0
        while temperature > stopping_temperature:
            i = i+1                                                                      #counts the number of iterations
            fp_new = PrimerDesign(PrimerDesign).get_primer_neighbour(fp_current)         #obtains new possible primer in neighbouring gene sequence
            rp_new = PrimerDesign(PrimerDesign).get_primer_neighbour(rp_current)         #obtains new possible primer in neighbouring gene sequence
            new_cost = PrimerDesign(PrimerDesign).cost_objective_function(fp_new, rp_new)#obtains new possible cost
            
            if new_cost < current_cost:                                                  #compares cost of new solution
                fp_current = fp_new                                                      
                rp_current = rp_new 
                current_cost = new_cost                                                  #accepts new solution with a lower cost than the current solution
            else:
                delta = new_cost-current_cost 
                acceptance_probability = np.exp(-delta/temperature)                      #else acceptance probability is calculated
                
                num = random.uniform(0, 1)                                               
                
                if acceptance_probability > num:                                         #accepts new solution if acceptance probability is less than random number between 0 and 1
                    fp_current = fp_new
                    rp_current = rp_new
                    current_cost = new_cost
            
            temperature = temperature*drop                                               #reduces temperature by decreasing factor
        print("Number of Iterations = {}".format(i))
        print("Objective Function Cost = {}".format(new_cost))
        print("Forward Primer = {}".format(fp_new))
        print("Reverse Primer = {}".format(rp_new))
        #PrimerDesign(PrimerDesign).cost_objective_function_info(fp_new, rp_new)
        return fp_new, rp_new, new_cost
    
    def get_primer_neighbour(self, sq):   
    
        length = random.randint(18, 22)                                                  #picks random length for neighbouring primer
        rdg = random.randint(-10, 10)                            
        ls = []
        dna = PrimerDesign(PrimerDesign).set_dna_sequence(dna_sequence)
        
        y = dna.find(sq)                                                                 #locates original primer in entire gene sequence
        y = y+rdg                                                                        #changes the specified location by a random integer between -10 and 10
        #print(y)
        for i in range(y, length+y):
            z = dna[i]
            ls.append(z)                                                                 #appends bases to new selected location of neighbouring primer
            seq = "".join(ls)                                                            #generates new primer in neighbouring gene sequence of random length
        return seq
        
PrimerDesign(PrimerDesign).func_simulated_annealing()

Number of Iterations = 149
Objective Function Cost = 9.0
Forward Primer = aagggccttgccgcaaagtgtg
Reverse Primer = gatacccaggaccaagccaca


('aagggccttgccgcaaagtgtg', 'gatacccaggaccaagccaca', 9.0)

* ** What were the simulation parameters that you used (in Step 1.1 of the algorithm)?**


** PrimerDesign(PrimerDesign).initial_temperature = 200 **

Higher temperature has larger probability of attaining global optimum. Thus, the temperature is extremely high in the beginning

** PrimerDesign(PrimerDesign).stopping_temperature = 0.1 **


** PrimerDesign(PrimerDesign).drop_fraction = 0.95 **

The Simulated Annealing algorithm works much better if the rate of cooling is slow, thus, a cooling factor close to 1 must be selected. However to optimize algorithm run time, we selected drop_fraction as 0.95.


* ** From the output of your code, how well does the final solution meet all the criteria?**

Comparing the output of the simulated annealing process to the random primer selection, the score of the objective function obtained for the random primer selected is higher as compared to the new primers generated by simulated annealing. Test cases have been printed below. This lower value for the cost objective function indicates that the simulated annealing program runs through iterations to pick the suitable primer with a low cost objective function, making it more effective.

* ** How many iterations were required to obtain this solution? **

Number of iterations = 149

* ** Will a brute-force algorithm (one that considers every single possible solution) require fewer or more iterations? Explain. **

A brute-force algorithm would require a lot more iterations and take a much longer time to return the result. Whereas the simulated annealing algorithm reduces the temperature by a certain drop fraction or decreasing temperature till it reaches the stoppping temperature, hence reducing the number of iterations greatly. This makes it a more efficient algorithm as it reduces the time brute-force algorithm would take otherwise.

### Instantiating Gene Sequences and Class

In [13]:
allele1 = '''1 gtccgggcag cccccggcgc agcgcggccg cagcagcctc cgccccccgc acggtgtgag
61 cgcccgacgc ggccgaggcg gccggagtcc cgagctagcc ccggcggccg ccgccgccca
121 gaccggacga caggccacct cgtcggcgtc cgcccgagtc cccgcctcgc cgccaacgcc
181 acaaccaccg cgcacggccc cctgactccg tccagtattg atcgggagag ccggagcgag
241 ctcttcgggg agcagcgatg cgaccctccg ggacggccgg ggcagcgctc ctggcgctgc
301 tggctgcgct ctgcccggcg agtcgggctc tggaggaaaa gaaagtttgc caaggcacga
361 gtaacaagct cacgcagttg ggcacttttg aagatcattt tctcagcctc cagaggatgt
421 tcaataactg tgaggtggtc cttgggaatt tggaaattac ctatgtgcag aggaattatg
481 atctttcctt cttaaagacc atccaggagg tggctggtta tgtcctcatt gccctcaaca
541 cagtggagcg aattcctttg gaaaacctgc agatcatcag aggaaatatg tactacgaaa
601 attcctatgc cttagcagtc ttatctaact atgatgcaaa taaaaccgga ctgaaggagc
661 tgcccatgag aaatttacag gaaatcctgc atggcgccgt gcggttcagc aacaaccctg
721 ccctgtgcaa cgtggagagc atccagtggc gggacatagt cagcagtgac tttctcagca
781 acatgtcgat ggacttccag aaccacctgg gcagctgcca aaagtgtgat ccaagctgtc
841 ccaatgggag ctgctggggt gcaggagagg agaactgcca gaaactgacc aaaatcatct
901 gtgcccagca gtgctccggg cgctgccgtg gcaagtcccc cagtgactgc tgccacaacc
961 agtgtgctgc aggctgcaca ggccccatgg agagcgactg cctggtctgc cgcaaattcc
1021 gagacgaagc cacgtgcaag gacacctgcc ccccactcat gctctacaac cccaccacgt
1081 accagatgga tgtgaacccc gagggcaaat acagctttgg tgccacctgc gtgaagaagt
1141 gtccccgtaa ttatgtggtg acagatcacg gctcgtgcgt ccgagcctgt ggggccgaca
1201 gctatgagat ggaggaagac ggcgtccgca agtgtaagaa gtgcgaaggg ccttgccgca
1261 aagtgtgtaa cggaataggt attggtgaat ttaaagactc actctccata aatgctacga
1321 atattaaaca cttcaaaaac tgcacctcca tcagtggcga tctccacatc ctgccggtgg
1381 catttagggg tgactccttc acacatactc ctcctctgga tccacaggaa ctggatattc
1441 tgaaaaccgt aaaggaaatc acagggtttt tgctgattca ggcttggcct gaaaacagga
1501 cggacctcca tgcctttgag aacctagaaa tcatacgcgg caggaccaag caacatggtc
1561 agttttctct tgcagtcgtc agcctgaaca taacatcctt gggattacgc tccctcaagg
1621 agataagtga tggagatgtg ataatttcag gaaacaaaaa tttgtgctat gcaaatacaa
1681 taaactggaa aaaactgttt gggacctccg gtcagaaaac caaaattata agcaacagag
1741 gtgaaaacag ctgcaaggcc acaggccagg tctgccatgc cttgtgctcc cccgagggct
1801 gctggggccc ggagcccagg gactgcgtct cttgccggaa tgtcagccga ggcagggaat
1861 gcgtggacaa gtgcaacctt ctggagggtg agccaaggga gtttgtggag aactctgagt
1921 gcatacagtg ccacccagag tgcctgcctc aggccatgaa catcacctgc acaggacggg
1981 gaccagacaa ctgtatccag tgtgcccact acattgacgg cccccactgc gtcaagacct
2041 gcccggcagg agtcatggga gaaaacaaca ccctggtctg gaagtacgca gacgcttgcc
2101 atgtgtgcca cctgtgccat ccaaactgca cctacggatg cactgggcca ggtcttgaag
2161 gctgtccaac gaatgggcct aagatcccgt ccatcgccac tgggatggtg ggggccctcc
2221 tcttgctgct ggtggtggcc ctggggatcg gcctcttcat gcgaaggcgc cacatcgttc
2281 ggaagcgcac gctgcggagg ctgctgcagg agagggagct tgtggagcct cttacaccca
2341 gtggagaagc tcccaaccaa gctctcttga ggatcttgaa ggaaactgaa ttcaaaaaga
2401 tcaaagtgct gggctccggt gcgttcggca cggtgtataa gggactctgg atcccagaag
2461 gtgagaaagt taaaattccc gtcgctatca aggaattaag agaagcaaca tctccgaaag
2521 ccaacaagga aatcctcgat gaagcctacg tgatggccag cgtggacaac ccccacgtgt
2581 gccgcctgct gggcatctgc ctcacctcca ccgtgcagct catcacgcag ctcatgccct
2641 tcggctgcct cctggactat gtccgggaac acaaagacaa tattggctcc cagtacctgc
2701 tcaactggtg tgtgcagatc gcaaagggca tgaactactt ggaggaccgt cgcttggtgc
2761 accgcgacct ggcagccagg aacgtactgg tgaaaacacc gcagcatgtc aagatcacag
2821 attttgggct ggccaaactg ctgggtgcgg aagagaaaga ataccatgca gaaggaggca
2881 aagtgcctat caagtggatg gcattggaat caattttaca cagaatctat acccaccaga
2941 gtgatgtctg gagctacggg gtgactgttt gggagttgat gacctttgga tccaagccat
3001 atgacggaat ccctgccagc gagatctcct ccatcctgga gaaaggagaa cgcctccctc
3061 agccacccat atgtaccatc gatgtctaca tgatcatggt caagtgctgg atgatagacg
3121 cagatagtcg cccaaagttc cgtgagttga tcatcgaatt ctccaaaatg gcccgagacc
3181 cccagcgcta ccttgtcatt cagggggatg aaagaatgca tttgccaagt cctacagact
3241 ccaacttcta ccgtgccctg atggatgaag aagacatgga cgacgtggtg gatgccgacg
3301 agtacctcat cccacagcag ggcttcttca gcagcccctc cacgtcacgg actcccctcc
3361 tgagctctct gagtgcaacc agcaacaatt ccaccgtggc ttgcattgat agaaatgggc
3421 tgcaaagctg tcccatcaag gaagacagct tcttgcagcg atacagctca gaccccacag
3481 gcgccttgac tgaggacagc atagacgaca ccttcctccc agtgcctgaa tacataaacc
3541 agtccgttcc caaaaggccc gctggctctg tgcagaatcc tgtctatcac aatcagcctc
3601 tgaaccccgc gcccagcaga gacccacact accaggaccc ccacagcact gcagtgggca
3661 accccgagta tctcaacact gtccagccca cctgtgtcaa cagcacattc gacagccctg
3721 cccactgggc ccagaaaggc agccaccaaa ttagcctgga caaccctgac taccagcagg
3781 acttctttcc caaggaagcc aagccaaatg gcatctttaa gggctccaca gctgaaaatg
3841 cagaatacct aagggtcgcg ccacaaagca gtgaatttat tggagcatga ccacggagga
3901 tagtatgagc cctaaaaatc cagactcttt cgatacccag gaccaagcca cagcaggtcc
3961 tccatcccaa cagccatgcc cgcattagct cttagaccca cagactggtt ttgcaacgtt
4021 tacaccgact agccaggaag tacttccacc tcgggcacat tttgggaagt tgcattcctt
4081 tgtcttcaaa ctgtgaagca tttacagaaa cgcatccagc aagaatattg tccctttgag
4141 cagaaattta tctttcaaag aggtatattt gaaaaaaaaa aaaagtatat gtgaggattt
4201 ttattgattg gggatcttgg agtttttcat tgtcgctatt gatttttact tcaatgggct
4261 cttccaacaa ggaagaagct tgctggtagc acttgctacc ctgagttcat ccaggcccaa
4321 ctgtgagcaa ggagcacaag ccacaagtct tccagaggat gcttgattcc agtggttctg
4381 cttcaaggct tccactgcaa aacactaaag atccaagaag gccttcatgg ccccagcagg
4441 ccggatcggt actgtatcaa gtcatggcag gtacagtagg ataagccact ctgtcccttc
4501 ctgggcaaag aagaaacgga ggggatggaa ttcttcctta gacttacttt tgtaaaaatg
4561 tccccacggt acttactccc cactgatgga ccagtggttt ccagtcatga gcgttagact
4621 gacttgtttg tcttccattc cattgttttg aaactcagta tgctgcccct gtcttgctgt
4681 catgaaatca gcaagagagg atgacacatc aaataataac tcggattcca gcccacattg
4741 gattcatcag catttggacc aatagcccac agctgagaat gtggaatacc taaggatagc
4801 accgcttttg ttctcgcaaa aacgtatctc ctaatttgag gctcagatga aatgcatcag
4861 gtcctttggg gcatagatca gaagactaca aaaatgaagc tgctctgaaa tctcctttag
4921 ccatcacccc aaccccccaa aattagtttg tgttacttat ggaagatagt tttctccttt
4981 tacttcactt caaaagcttt ttactcaaag agtatatgtt ccctccaggt cagctgcccc
5041 caaaccccct ccttacgctt tgtcacacaa aaagtgtctc tgccttgagt catctattca
5101 agcacttaca gctctggcca caacagggca ttttacaggt gcgaatgaca gtagcattat
5161 gagtagtgtg gaattcaggt agtaaatatg aaactagggt ttgaaattga taatgctttc
5221 acaacatttg cagatgtttt agaaggaaaa aagttccttc ctaaaataat ttctctacaa
5281 ttggaagatt ggaagattca gctagttagg agcccacctt ttttcctaat ctgtgtgtgc
5341 cctgtaacct gactggttaa cagcagtcct ttgtaaacag tgttttaaac tctcctagtc
5401 aatatccacc ccatccaatt tatcaaggaa gaaatggttc agaaaatatt ttcagcctac
5461 agttatgttc agtcacacac acatacaaaa tgttcctttt gcttttaaag taatttttga
5521 ctcccagatc agtcagagcc cctacagcat tgttaagaaa gtatttgatt tttgtctcaa
5581 tgaaaataaa actatattca tttccactct attatgctct caaatacccc taagcatcta
5641 tactagcctg gtatgggtat gaaagataca aagataaata aaacatagtc cctgattcta
5701 agaaattcac aatttagcaa aggaaatgga ctcatagatg ctaaccttaa aacaacgtga
5761 caaatgccag acaggaccca tcagccaggc actgtgagag cacagagcag ggaggttggg
5821 tcctgcctga ggagacctgg aagggaggcc tcacaggagg atgaccaggt ctcagtcagc
5881 ggggaggtgg aaagtgcagg tgcatcaggg gcaccctgac cgaggaaaca gctgccagag
5941 gcctccactg ctaaagtcca cataaggctg aggtcagtca ccctaaacaa cctgctccct
6001 ctaagccagg ggatgagctt ggagcatccc acaagttccc taaaagttgc agcccccagg
6061 gggattttga gctatcatct ctgcacatgc ttagtgagaa gactacacaa catttctaag
6121 aatctgagat tttatattgt cagttaacca ctttcattat tcattcacct caggacatgc
6181 agaaatattt cagtcagaac tgggaaacag aaggacctac attctgctgt cacttatgtg
6241 tcaagaagca gatgatcgat gaggcaggtc agttgtaagt gagtcacatt gtagcattaa
6301 attctagtat ttttgtagtt tgaaacagta acttaataaa agagcaaaag ctaaaaaaaa
6361 aaaaaaaaa
'''
allele2 = '''1 gtccgggcag cccccggcgc agcgcggccg cagcagcctc cgccccccgc acggtgtgag
61 cgcccgacgc ggccgaggcg gccggagtcc cgagctagcc ccggcggccg ccgccgccca
121 gaccggacga caggccacct cgtcggcgtc cgcccgagtc cccgcctcgc cgccaacgcc
181 acaaccaccg cgcacggccc cctgactccg tccagtattg atcgggagag ccggagcgag
241 ctcttcgggg agcagcgatg cgaccctccg ggacggccgg ggcagcgctc ctggcgctgc
301 tggctgcgct ctgcccggcg agtcgggctc tggaggaaaa gaaagtttgc caaggcacga
361 gtaacaagct cacgcagttg ggcacttttg aagatcattt tctcagcctc cagaggatgt
421 tcaataactg tgaggtggtc cttgggaatt tggaaattac ctatgtgcag aggaattatg
481 atctttcctt cttaaagacc atccaggagg tggctggtta tgtcctcatt gccctcaaca
541 cagtggagcg aattcctttg gaaaacctgc agatcatcag aggaaatatg tactacgaaa
601 attcctatgc cttagcagtc ttatctaact atgatgcaaa taaaaccgga ctgaaggagc
661 tgcccatgag aaatttacag gaaatcctgc atggcgccgt gcggttcagc aacaaccctg
721 ccctgtgcaa cgtggagagc atccagtggc gggacatagt cagcagtgac tttctcagca
781 acatgtcgat ggacttccag aaccacctgg gcagctgcca aaagtgtgat ccaagctgtc
841 ccaatgggag ctgctggggt gcaggagagg agaactgcca gaaactgacc aaaatcatct
901 gtgcccagca gtgctccggg cgctgccgtg gcaagtcccc cagtgactgc tgccacaacc
961 agtgtgctgc aggctgcaca ggcccccggg agagcgactg cctggtctgc cgcaaattcc
1021 gagacgaagc cacgtgcaag gacacctgcc ccccactcat gctctacaac cccaccacgt
1081 accagatgga tgtgaacccc gagggcaaat acagctttgg tgccacctgc gtgaagaagt
1141 gtccccgtaa ttatgtggtg acagatcacg gctcgtgcgt ccgagcctgt ggggccgaca
1201 gctatgagat ggaggaagac ggcgtccgca agtgtaagaa gtgcgaaggg ccttgccgca
1261 aagtgtgtaa cggaataggt attggtgaat ttaaagactc actctccata aatgctacga
1321 atattaaaca cttcaaaaac tgcacctcca tcagtggcga tctccacatc ctgccggtgg
1381 catttagggg tgactccttc acacatactc ctcctctgga tccacaggaa ctggatattc
1441 tgaaaaccgt aaaggaaatc acagggtttt tgctgattca ggcttggcct gaaaacagga
1501 cggacctcca tgcctttgag aacctagaaa tcatacgcgg caggaccaag caacatggtc
1561 agttttctct tgcagtcgtc agcctgaaca taacatcctt gggattacgc tccctcaagg
1621 agataagtga tggagatgtg ataatttcag gaaacaaaaa tttgtgctat gcaaatacaa
1681 taaactggaa aaaactgttt gggacctccg gtcagaaaac caaaattata agcaacagag
1741 gtgaaaacag ctgcaaggcc acaggccagg tctgccatgc cttgtgctcc cccgagggct
1801 gctggggccc ggagcccagg gactgcgtct cttgccggaa tgtcagccga ggcagggaat
1861 gcgtggacaa gtgcaacctt ctggagggtg agccaaggga gtttgtggag aactctgagt
1921 gcatacagtg ccacccagag tgcctgcctc aggccatgaa catcacctgc acaggacggg
1981 gaccagacaa ctgtatccag tgtgcccact acattgacgg cccccactgc gtcaagacct
2041 gcccggcagg agtcatggga gaaaacaaca ccctggtctg gaagtacgca gacgccggcc
2101 atgtgtgcca cctgtgccat ccaaactgca cctacggatg cactgggcca ggtcttgaag
2161 gctgtccaac gaatgggcct aagatcccgt ccatcgccac tgggatggtg ggggccctcc
2221 tcttgctgct ggtggtggcc ctggggatcg gcctcttcat gcgaaggcgc cacatcgttc
2281 ggaagcgcac gctgcggagg ctgctgcagg agagggagct tgtggagcct cttacaccca
2341 gtggagaagc tcccaaccaa gctctcttga ggatcttgaa ggaaactgaa ttcaaaaaga
2401 tcaaagtgct gggctccggt gcgttcggca cggtgtataa gggactctgg atcccagaag
2461 gtgagaaagt taaaattccc gtcgctatca aggaattaag agaagcaaca tctccgaaag
2521 ccaacaagga aatcctcgat gaagcctacg tgatggccag cgtggacaac ccccacgtgt
2581 gccgcctgct gggcatctgc ctcacctcca ccgtgcagct catcacgcag ctcatgccct
2641 tcggctgcct cctggactat gtccgggaac acaaagacaa tattggctcc cagtacctgc
2701 tcaactggtg tgtgcagatc gcaaagggca tgaactactt ggaggaccgt cgcttggaaa
2761 accgcgacct ggcagccagg aacgtactgg tgaaaacacc gcagcatgtc aagatcacag
2821 attttgggct ggccaaactg ctgggtgcgg aagagaaaga ataccatgca gaaggaggca
2881 aagtgcctat caagtggatg gcattggaat caattttaca cagaatctat acccaccaga
2941 gtgatgtctg gagctacggg gtgactgttt gggagttgat gacctttgga tccaagccat
3001 atgacggaat ccctgccagc gagatctcct ccatcctgga gaaaggagaa cgcctccctc
3061 agccacccat atgtaccatc gatgtctaca tgatcatggt caagtgctgg atgatagacg
3121 cagatagtcg cccaaagttc cgtgagttga tcatcgaatt ctccaaaatg gcccgagacc
3181 cccagcgcta ccttgtcatt cagggggatg aaagaatgca tttgccaagt cctacagact
3241 ccaacttcta ccgtgccctg atggatgaag aagacatgga cgacgtggtg gatgccgacg
3301 agtacctcat cccacagcag ggcttcttca gcagcccctc cacgtcacgg actcccctcc
3361 tgagctctct gagtgcaacc agcaacaatt ccaccgtggc ttgcattgat agaaatgggc
3421 tgcaaagctg tcccatcaag gaagacagct tcttgcagcg atacagctca gaccccacag
3481 gcgccttgac tgaggacagc atagacgaca ccttcctccc agtgcctgaa tacataaacc
3541 agtccgttcc caaaaggccc gctggctctg tgcagaatcc tgtctatcac aatcagcctc
3601 tgaaccccgc gcccagcaga gacccacact accaggaccc ccacagcact gcagtgggca
3661 accccgagta tctcaacact gtccagccca cctgtgtcaa cagcacattc gacagccctg
3721 cccactgggc ccagaaaggc agccaccaaa ttagcctgga caaccctgac taccagcagg
3781 acttctttcc caaggaagcc aagccaaatg gcatctttaa gggctccaca gctgaaaatg
3841 cagaatacct aagggtcgcg ccacaaagca gtgaatttat tggagcatga ccacggagga
3901 tagtatgagc cctaaaaatc cagactcttt cgatacccag gaccaagcca cagcaggtcc
3961 tccatcccaa cagccatgcc cgcattagct cttagaccca cagactggtt ttgcaacgtt
4021 tacaccgact agccaggaag cgtttccacc tcgggcacat tttgggaagt tgcattcctt
4081 tgtcttcaaa ctgtgaagca tttacagaaa cgcatccagc aagaatattg tccctttgag
4141 cagaaattta tctttcaaag aggtatattt gaaaaaaaaa aaaagtatat gtgaggattt
4201 ttattgattg gggatcttgg agtttttcat tgtcgctatt gatttttact tcaatgggct
4261 cttccaacaa ggaagaagct tgctggtagc acttgctacc ctgagttcat ccaggcccaa
4321 ctgtgagcaa ggagcacaag ccacaagtct tccagaggat gcttgattcc agtggttctg
4381 cttcaaggct tccactgcaa aacactaaag atccaagaag gccttcatgg ccccagcagg
4441 ccggatcggt actgtatcaa gtcatggcag gtacagtagg ataagccact ctgtcccttc
4501 ctgggcaaag aagaaacgga ggggatggaa ttcttcctta gacttacttt tgtaaaaatg
4561 tccccacggt acttactccc cactgatgga ccagtggttt ccagtcatga gcgttagact
4621 gacttgtttg tcttccattc cattgttttg aaactcagta tgctgcccct gtcttgctgt
4681 catgaaatca gcaagagagg atgacacatc aaataataac tcggattcca gcccacattg
4741 gattcatcag catttggacc aatagcccac agctgagaat gtggaatacc taaggatagc
4801 accgcttttg ttctcgcaaa aacgtatctc ctaatttgag gctcagatga aatgcatcag
4861 gtcctttggg gcatagatca gaagactaca aaaatgaagc tgctctgaaa tctcctttag
4921 ccatcacccc aaccccccaa aattagtttg tgttacttat ggaagatagt tttctccttt
4981 tacttcactt caaaagcttt ttactcaaag agtatatgtt ccctccaggt cagctgcccc
5041 caaaccccct ccttacgctt tgtcacacaa aaagtgtctc tgccttgagt catctattca
5101 agcacttaca gctctggcca caacagggca ttttacaggt gcgaatgaca gtagcattat
5161 gagtagtgtg gaattcaggt agtaaatatg aaactagggt ttgaaattga taatgctttc
5221 acaacatttg cagatgtttt agaaggaaaa aagttccttc ctaaaataat ttctctacaa
5281 aacgaagatt ggaagattca gctagttagg agcccacctt ttttcctaat ctgtgtgtgc
5341 cctgtaacct gactggttaa cagcagtcct ttgtaaacag tgttttaaac tctcctagtc
5401 aatatccacc ccatccaatt tatcaaggaa gaaatggttc agaaaatatt ttcagcctac
5461 agttatgttc agtcacacac acatacaaaa tgttcctttt gcttttaaag taatttttga
5521 ctcccagatc agtcagagcc cctacagcat tgttaagaaa gtatttgatt tttgtctcaa
5581 tgaaaataaa actatattca tttccactct attatgctct caaatacccc taagcatcta
5641 tactagcctg gtatgggtat gaaagataca aagataaata aaacatagtc cctgattcta
5701 agaaattcac aatttagcaa aggaaatgga ctcatagatg ctaaccttaa aacaacgtga
5761 caaatgccag acaggaccca tcagccaggc actgtgagag cacagagcag ggaggttggg
5821 tcctgcctga ggagacctgg aagggaggcc tcacaggagg atgaccaggt ctcagtcagc
5881 ggggaggtgg aaagtgcagg tgcatcaggg gcaccctgac cgaggaaaca gctgccagag
5941 gcctccactg ctaaagtcca cataaggctg aggtcagtca ccctaaacaa cctgctccct
6001 ctaagccagg ggatgagctt ggagcatccc acaagttccc taaaagttgc agcccccagg
6061 gggattttga gctatcatct ctgcacatgc ttagtgagaa gactacacaa catttctaag
6121 aatctgagat tttatattgt cagttaacca ctttcattat tcattcacct caggacatgc
6181 agaaatattt cagtcagaac tgggaaacag aaggacctac attctgctgt cacttatgtg
6241 tcaagaagca gatgatcgat gaggcaggtc agttgtaagt gagtcacatt gtagcattaa
6301 attctagtat ttttgtagtt tgaaacagta acttaataaa agagcaaaag ctaaaaaaaa
6361 aaaaaaaaa

'''

dna_sequence = allele1

### For allele1:

Primer Design Class has been instantiated as object a.
The initialized parameters can be specified correctly and altered using the command

* a.parameter/property = value

Any function present in the class can be called using the command

* a.function_name(fp/rp)
* a.cost_name(fp/rp)

The (a.cost_objective_function_info) function displays a report of the random forward and backward primers selected and their suitability.

In [14]:
a = PrimerDesign(allele1)
a.fp_start = 333
a.fp_end = 1500
a.min_temp = 50
a.penalty_cg = 2
a.cost_objective_function_info(fp, rp)


                      Selected Primers                      
         ===Forward Primer=== aggccagtcttttggtttta          


Criterion                  Value         Cost Function Score
------------------------------------------------------------
Length                       20                            0
Annealing Temperature        56                            0
% cg Content                40.0                         0.0
% at Content                60.0        
Runs                         0                             0
Repeats                      0                             0
Specificity                                                0


         ===Reverse Primer=== tttcctcttgcggagggagt          


Criterion                  Value         Cost Function Score
------------------------------------------------------------
Length                       20                            0
Annealing Temperature        62                            0
% cg Content                55.0      

### For allele2:

Primer Design Class has been instantiated as object b.
The initialized parameters can be specified correctly and adjusted using the command

* b.parameter/property = value

Any function present in the class can be called using the command

* b.function_name(fp/rp)
* b.cost_name(fp/rp)

The (b.cost_objective_function_info) function displays a report of the random forward and backward primers selected and their suitability.

In [15]:
b = PrimerDesign(allele2)
b.rp_start = 2000
b.rp_end = 3000
b.repeat_threshold = 1
b.penalty_specificity = 2
b.cost_objective_function_info(fp, rp)

                      Selected Primers                      
         ===Forward Primer=== aggccagtcttttggtttta          


Criterion                  Value         Cost Function Score
------------------------------------------------------------
Length                       20                            0
Annealing Temperature        56                            0
% cg Content                40.0                         0.0
% at Content                60.0        
Runs                         0                             0
Repeats                      0                             0
Specificity                                                0


         ===Reverse Primer=== tttcctcttgcggagggagt          


Criterion                  Value         Cost Function Score
------------------------------------------------------------
Length                       20                            0
Annealing Temperature        62                            0
% cg Content                55.0      

###  Simulated Annealing for allele 1

As this is a probabilistic algorithm, we run it more than once to get a reasonable optimal solution.

In [16]:
x = PrimerDesign(allele1)
print("trial 1 = {} \n".format(x.func_simulated_annealing()))
print("trial 2 = {} \n".format(x.func_simulated_annealing()))
'''trial 3'''
x_fp, x_rp, x_cost = x.func_simulated_annealing()
x.cost_objective_function_info(x_fp, x_rp)

Number of Iterations = 149
Objective Function Cost = 9.0
Forward Primer = acataacatccttgggattacg
Reverse Primer = aggaagtacttccacctcgggc
trial 1 = ('acataacatccttgggattacg', 'aggaagtacttccacctcgggc', 9.0) 

Number of Iterations = 149
Objective Function Cost = 7.0
Forward Primer = aaactgcacctccatcagtggc
Reverse Primer = gaaatgggctgcaaagctgt
trial 2 = ('aaactgcacctccatcagtggc', 'gaaatgggctgcaaagctgt', 7.0) 

Number of Iterations = 149
Objective Function Cost = 0.0
Forward Primer = aaaacaggacggacctccat
Reverse Primer = ctttcccaaggaagccaagc
                      Selected Primers                      
         ===Forward Primer=== ttttgtcctgcctggaggta          


Criterion                  Value         Cost Function Score
------------------------------------------------------------
Length                       20                            0
Annealing Temperature        60                            0
% cg Content                50.0                         0.0
% at Content                

### Simulated Annealing for allele 2

As this is a probabilistic algorithm, we run it more than once to get a reasonable optimal solution.

In [17]:
y = PrimerDesign(allele2)
print("trial 1 = {} \n".format(y.func_simulated_annealing()))
print("trial 2 = {} \n".format(y.func_simulated_annealing()))
'''trial 3'''
y_fp, y_rp, y_cost = y.func_simulated_annealing()
y.cost_objective_function_info(y_fp, y_rp)

Number of Iterations = 149
Objective Function Cost = 7.0
Forward Primer = tccttgggattacgctccctca
Reverse Primer = tagacgcagatagtcgccc
trial 1 = ('tccttgggattacgctccctca', 'tagacgcagatagtcgccc', 7.0) 

Number of Iterations = 149
Objective Function Cost = 14.16
Forward Primer = aaaactgtttgggacctccggt
Reverse Primer = cactgggcccagaaaggcagcc
trial 2 = ('aaaactgtttgggacctccggt', 'cactgggcccagaaaggcagcc', 14.16) 

Number of Iterations = 149
Objective Function Cost = 0.06
Forward Primer = tgcagtcgtcagcctgaaca
Reverse Primer = gtgggcaaccccgagtatc
                      Selected Primers                      
         ===Forward Primer=== acgtcagcagtcggacttgt          


Criterion                  Value         Cost Function Score
------------------------------------------------------------
Length                       20                            0
Annealing Temperature        62                            0
% cg Content                55.0                         0.0
% at Content              

### Test Cases

In [18]:
test1 = PrimerDesign('cattaaaaatacgaaaaaagtcat')
print(test1.func_length('cattaaaaatacgaaaaaagtcat'))
print(test1.func_cg_fraction('cattaaaaatacgaaaaaagtcat'))
print(test1.func_temperature('cattaaaaatacgaaaaaagtcat'))
print(test1.func_count_runs('cattaaaaatacgaaaaaagtcat'))
print(test1.func_count_repeats('cattaaaaatacgaaaaaagtcat'))

24
5/24
58
2
0


In [19]:
test2 = PrimerDesign('atatatatattttatatataa')
print(test2.func_length('atatatatattttatatataa'))
print(test2.func_cg_fraction('atatatatattttatatataa'))
print(test2.func_temperature('atatatatattttatatataa'))
print(test2.func_count_runs('atatatatattttatatataa'))
print(test2.func_count_repeats('atatatatattttatatataa'))

21
0
42
0
12


In [20]:
test3 = PrimerDesign('')
print(test3.func_length(''))
print(test3.func_cg_fraction(''))
print(test3.func_temperature(''))
print(test3.func_count_runs(''))
print(test3.func_count_repeats(''))

0
0
0
0
0
